In [2]:
from ultralytics import YOLO
import ultralytics
import os
import numpy as np
from pathlib import Path
#set visible cuda

os.environ['CUDA_VISIBLE_DEVICES'] = '1'


# Load a pretrained YOLOv8n model
model = YOLO('/Data4/student_zhihan_data/source_code/yolo/ultralytics/runs/detect/GC10-DET_brightness_0 detect by yolov8n with dropout(p=0.1)/weights/best.pt')

# Define path to directory containing images and videos for inference
source = '/Data4/student_zhihan_data/data/GC10-DET/test/images'


# Run inference on the source
results = model([os.path.join(source, i) for i in os.listdir(source)])# generator of Results objects


0: 640x640 1 crescent_gap, 1 welding_line, 1: 640x640 1 water_spot, 2: 640x640 1 oil_spot, 3: 640x640 1 water_spot, 4: 640x640 4 crescent_gaps, 1 welding_line, 5: 640x640 1 silk_spot, 6: 640x640 3 oil_spots, 3 punching_holes, 2 welding_lines, 7: 640x640 2 water_spots, 8: 640x640 1 waist_folding, 9: 640x640 1 waist_folding, 10: 640x640 1 crescent_gap, 1 welding_line, 11: 640x640 2 crescent_gaps, 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 2 silk_spots, 15: 640x640 1 water_spot, 16: 640x640 2 oil_spots, 17: 640x640 1 silk_spot, 18: 640x640 2 oil_spots, 19: 640x640 1 crescent_gap, 2 welding_lines, 20: 640x640 1 crescent_gap, 1 welding_line, 21: 640x640 (no detections), 22: 640x640 1 punching_hole, 23: 640x640 1 punching_hole, 1 welding_line, 24: 640x640 1 punching_hole, 25: 640x640 1 silk_spot, 26: 640x640 1 punching_hole, 3 welding_lines, 27: 640x640 1 water_spot, 28: 640x640 1 water_spot, 29: 640x640 (no detections), 30: 640x640 1 silk_spot, 1 water_spot, 31: 

In [3]:
from ultralytics.utils.metrics import compute_ap
from ultralytics.engine.validator import BaseValidator
from ultralytics.utils.metrics import box_iou, Metric, DetMetrics
import numpy as np
import torch
import pandas as pd

def smooth(y, f=0.05):
    """Box filter of fraction f."""
    nf = round(len(y) * f * 2) // 2 + 1  # number of filter elements (must be odd)
    p = np.ones(nf // 2)  # ones padding
    yp = np.concatenate((p * y[0], y, p * y[-1]), 0)  # y padded
    return np.convolve(yp, np.ones(nf) / nf, mode="valid")  # y-smoothed

def ap_per_class(
    tp, conf, pred_cls, target_cls, plot=False, on_plot=None, save_dir=Path(), names=(), eps=1e-16, prefix=""
):
    """
    Computes the average precision per class for object detection evaluation.

    Args:
        tp (np.ndarray): Binary array indicating whether the detection is correct (True) or not (False).
        conf (np.ndarray): Array of confidence scores of the detections.
        pred_cls (np.ndarray): Array of predicted classes of the detections.
        target_cls (np.ndarray): Array of true classes of the detections.
        plot (bool, optional): Whether to plot PR curves or not. Defaults to False.
        on_plot (func, optional): A callback to pass plots path and data when they are rendered. Defaults to None.
        save_dir (Path, optional): Directory to save the PR curves. Defaults to an empty path.
        names (tuple, optional): Tuple of class names to plot PR curves. Defaults to an empty tuple.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-16.
        prefix (str, optional): A prefix string for saving the plot files. Defaults to an empty string.

    Returns:
        (tuple): A tuple of six arrays and one array of unique classes, where:
            tp (np.ndarray): True positive counts at threshold given by max F1 metric for each class.Shape: (nc,).
            fp (np.ndarray): False positive counts at threshold given by max F1 metric for each class. Shape: (nc,).
            p (np.ndarray): Precision values at threshold given by max F1 metric for each class. Shape: (nc,).
            r (np.ndarray): Recall values at threshold given by max F1 metric for each class. Shape: (nc,).
            f1 (np.ndarray): F1-score values at threshold given by max F1 metric for each class. Shape: (nc,).
            ap (np.ndarray): Average precision for each class at different IoU thresholds. Shape: (nc, 10).
            unique_classes (np.ndarray): An array of unique classes that have data. Shape: (nc,).
            p_curve (np.ndarray): Precision curves for each class. Shape: (nc, 1000).
            r_curve (np.ndarray): Recall curves for each class. Shape: (nc, 1000).
            f1_curve (np.ndarray): F1-score curves for each class. Shape: (nc, 1000).
            x (np.ndarray): X-axis values for the curves. Shape: (1000,).
            prec_values: Precision values at mAP@0.5 for each class. Shape: (nc, 1000).
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes, nt = np.unique(target_cls, return_counts=True)
    nc = unique_classes.shape[0]  # number of classes, number of detections

    # Create Precision-Recall curve and compute AP for each class
    x, prec_values = np.linspace(0, 1, 1000), []

    # Average precision, precision and recall curves
    ap, p_curve, r_curve = np.zeros((nc, tp.shape[1])), np.zeros((nc, 1000)), np.zeros((nc, 1000))
    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        n_l = nt[ci]  # number of labels
        n_p = i.sum()  # number of predictions
        if n_p == 0 or n_l == 0:
            continue

        # Accumulate FPs and TPs
        fpc = (1 - tp[i]).cumsum(0)
        tpc = tp[i].cumsum(0)

        # Recall
        recall = tpc / (n_l + eps)  # recall curve
        r_curve[ci] = np.interp(-x, -conf[i], recall[:, 0], left=0)  # negative x, xp because xp decreases

        # Precision
        precision = tpc / (tpc + fpc)  # precision curve
        p_curve[ci] = np.interp(-x, -conf[i], precision[:, 0], left=1)  # p at pr_score

        # AP from recall-precision curve
        for j in range(tp.shape[1]):
            ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])
            if plot and j == 0:
                prec_values.append(np.interp(x, mrec, mpre))  # precision at mAP@0.5

    prec_values = np.array(prec_values)  # (nc, 1000)

    # Compute F1 (harmonic mean of precision and recall)
    f1_curve = 2 * p_curve * r_curve / (p_curve + r_curve + eps)
    f2_curve = (5 * p_curve * r_curve) / (4 * p_curve + r_curve + eps)
    # names = [v for k, v in names.items() if k in unique_classes]  # list: only classes that have data
    # names = dict(enumerate(names))  # to dict
    # if plot:
    #     plot_pr_curve(x, prec_values, ap, save_dir / f"{prefix}PR_curve.png", names, on_plot=on_plot)
    #     plot_mc_curve(x, f1_curve, save_dir / f"{prefix}F1_curve.png", names, ylabel="F1", on_plot=on_plot)
    #     plot_mc_curve(x, p_curve, save_dir / f"{prefix}P_curve.png", names, ylabel="Precision", on_plot=on_plot)
    #     plot_mc_curve(x, r_curve, save_dir / f"{prefix}R_curve.png", names, ylabel="Recall", on_plot=on_plot)

    i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
    p, r, f1, f2 = p_curve[:, i], r_curve[:, i], f1_curve[:, i], f2_curve[:, i]  # max-F1 precision, recall, F1 values
    tp = (r * nt).round()  # true positives
    fp = (tp / (p + eps) - tp).round()  # false positives
    
    ap_50 = ap[:, 0].mean()
    ap_50_95 = ap.mean()
    
    return tp, fp, p, r, f1, f2, ap_50, ap_50_95, unique_classes.astype(int), p_curve, r_curve, f1_curve, f2_curve, x, prec_values


In [ ]:
def compute_f2(source, model):
    results = model([os.path.join(source, i) for i in os.listdir(source)])
    validator = BaseValidator()
    validator.iouv = torch.arange(0.5, 1, 0.05)
    
    df = pd.DataFrame(columns=['img_name', 'p', 'r', 'f1', 'f2', 'ap_50', 'ap_50_95', 'conf', 'pred_cls', 'target_cls'])

    for result in results:
        metric = DetMetrics()
        label_path = result.path.replace('images', 'labels')[:-4] + '.txt'
        label = torch.from_numpy(np.loadtxt(label_path))
        # turn label into (x1, y1, x2, y2) format
        
        if len(label) == 0:
            continue
        
        if label.dim() > 1:
            x1 = label[:, 1] - label[:, 3] / 2
            y1 = label[:, 2] - label[:, 4] / 2
            x2 = label[:, 1] + label[:, 3] / 2
            y2 = label[:, 2] + label[:, 4] / 2
            label = torch.stack((label[:, 0], x1, y1, x2, y2), 1)
            iou = box_iou(label[:, 1:].to("cuda:0"), result.boxes.xyxyn.to("cuda:0"))
            tp = validator.match_predictions(result.boxes.data[:,-1], label[:, 0].to("cuda:0"), iou)
            
            # update metric
            tp = tp.detach().cpu().numpy()
            conf = result.boxes.conf.detach().cpu().numpy()
            pre_cls = result.boxes.cls.detach().cpu().numpy()
            target_cls = label[:, 0].detach().cpu().numpy()
        
        elif label.dim() == 1:
            x1 = label[1] - label[3] / 2
            y1 = label[2] - label[4] / 2
            x2 = label[1] + label[3] / 2
            y2 = label[2] + label[4] / 2
            label = torch.tensor([label[0], x1, y1, x2, y2])
            iou = box_iou(label[1:].to("cuda:0").reshape(1, -1), result.boxes.xyxyn.to("cuda:0"))
            tp = validator.match_predictions(result.boxes.data[:,-1], label[0].to("cuda:0").unsqueeze(0), iou)
        
            # update metric
            tp = tp.detach().cpu().numpy()
            conf = result.boxes.conf.detach().cpu().numpy()
            pre_cls = result.boxes.cls.detach().cpu().numpy()
            target_cls = label[0].unsqueeze(0).detach().cpu().numpy()
            
        _, _, p, r, f1, f2, ap_50, ap_50_95, unique_classes, p_curve, r_curve, f1_curve, f2_curve, x, prec_values = ap_per_class(tp, conf, pre_cls, target_cls)
        print(f2.mean(), unique_classes)
        
        # update df
        df.loc[len(df)] = [result.path, p, r, f1, f2, ap_50, ap_50_95, conf, pre_cls, target_cls]

    df.to_csv(f'/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/F2_Record/{source.split("/")[-3]}_new.csv', index=False, header=True)
    return df
  

for i in os.listdir('/Data4/student_zhihan_data/data'):
    if i[-3:] != 'csv' and i != 'NEU-DET' and i != 'data.zip':
        source = os.path.join('/Data4/student_zhihan_data/data', i, 'test/images')
        df = compute_f2(source, model)

In [4]:
# input: uncertainty * 3, proposed * score, NIQE, BRISQUE
# output: quality score 

F2_dir = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/F2_Record'
Proposed_dir = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Score_Record'
Uncertainty_dir = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record'

# concat all csv
dfs = []
for idx, dir in enumerate([F2_dir, Proposed_dir, Uncertainty_dir]):
    files = os.listdir(dir)
    
    if idx == 0:
        continue     
        # df.to_csv(f'{dir}_new.csv', index=False, header=True)
        for file in files:
            if file[-7:] == 'new.csv':
                df = pd.concat([df, pd.read_csv(os.path.join(dir, file))], ignore_index=True)
            
    elif idx == 1:
        df = pd.DataFrame(columns=['dataset','img_name','p', 'r', 'f1', 'f2', 'ap_50', 'ap_50_95', 'conf', 'pred_cls', 'target_cls'])
        for file in files:
            if file[-3:] == 'csv':
                df = pd.concat([df, pd.read_csv(os.path.join(dir, file))], ignore_index=True)
        df['img_name'] = df['dataset'] + '/images/' + df['img_name']
        df = df.drop(columns=['dataset'])
        # df.to_csv(f'{dir}.csv', index=False, header=True)
    else:
        for file in files:
            if file[-3:] == 'csv':
                df = pd.concat([df, pd.read_csv(os.path.join(dir, file))], ignore_index=True)
        df['img_name'] = df['dataset'] + '/test/images/' + df['img_name']
        df = df.drop(columns=['dataset'])
        # df.to_csv(f'{dir}.csv', index=False, header=True)
    dfs.append(df)

In [23]:
#combine
df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv')
df['img_name'] = df['dataset'] + '/test/images/' + df['img_name']
df = df.drop(columns=['dataset'])
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv', index=False, header=True)

df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv')
df['img_name'] = df['dataset'] + '/images/' + df['img_name']
df = df.drop(columns=['dataset'])
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv', index=False, header=True)

In [19]:
files = os.listdir(Uncertainty_dir)
df = pd.DataFrame(columns=['dataset','img_name','objectness_uncertainty','objectness_entropy','weighted_variance_sum','weighted_entropy'])
for file in files:
    if file[-3:] == 'csv':
        tmp = pd.read_csv(os.path.join(Uncertainty_dir, file))
        #add header to tmp
        tmp.columns = ['dataset','img_name','objectness_uncertainty','objectness_entropy','weighted_variance_sum','weighted_entropy']
        df = pd.concat([df, tmp], ignore_index=True)

df.to_csv(f'Uncertainty_Record.csv', index=False, header=True)

/tmp/ipykernel_2732998/2401473654.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp], ignore_index=True)


In [18]:
files = os.listdir(Proposed_dir)
df = pd.DataFrame(columns=['dataset', 'img_name', 'visibility', 'exposure'])
for file in files:
    if file[-3:] == 'csv':
        tmp = pd.read_csv(os.path.join(Proposed_dir, file))
        #add header to tmp
        tmp.columns = ['dataset', 'img_name', 'visibility', 'exposure']
        df = pd.concat([df, tmp], ignore_index=True)

df.to_csv(f'Proposed_Record.csv', index=False, header=True)

/tmp/ipykernel_2732998/2492081121.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp], ignore_index=True)


In [5]:
dfs = []
for idx, dir in enumerate(['/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/F2_Record_new.csv',
                          '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv',
                           '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv']):
    df = pd.read_csv(dir)
    dfs.append(df)
 
merged_df = pd.merge(dfs[0], dfs[1], on=['img_name'])
merged_df = pd.merge(merged_df, dfs[2], on=['img_name'])
    
    
# columns = set()
# for df in dfs:
#     columns = columns.union(set(df.columns))

# df = pd.DataFrame(columns=list(columns))
# img_name_union = set(dfs[0]['img_name'])

# for tmp in dfs:
#     img_name_union = img_name_union.intersection(set(tmp['img_name']))

# for tmp in dfs:
#     merged_df = pd.merge(df, tmp, on=['img_name'])
    

In [6]:
merged_df['weighted_entropy'] = merged_df['weighted_variance_sum']
merged_df['weighted_variance_sum'] = merged_df['objectness_entropy']
merged_df.drop(columns=['objectness_entropy'], inplace=True)
merged_df.describe()

,ap_50,ap_50_95,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy
count,7064.000000,7064.000000,7.064000e+03,7064.000000,7.064000e+03,7064.000000,7064.000000
mean,0.475281,0.239499,2.669650e-01,0.202524,6.347572e-03,0.001332,0.144991
std,0.466809,0.275075,2.080242e-01,0.260773,4.891388e-03,0.004060,0.071955
min,0.000000,0.000000,3.222425e-07,0.000000,3.902979e-08,0.000000,0.050507
25%,0.000000,0.000000,9.706808e-02,0.006841,2.335405e-03,0.000038,0.096591
50%,0.497500,0.099500,2.238936e-01,0.075973,5.465690e-03,0.000169,0.105462
75%,0.995000,0.460437,3.947939e-01,0.326185,9.125920e-03,0.000804,0.205264
max,0.995000,0.995000,9.695124e-01,0.999207,2.989884e-02,0.056565,0.479202


In [27]:
merged_df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/merged_new.csv', index=False, header=True)

In [94]:
df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv')
a =  df['img_name'].apply(lambda x: '/'.join(x.split('/')[:-2]))
b = df['img_name'].apply(lambda x: '/'.join(x.split('/')[-2:]))

#concat a, '/images/, b to a new column
df['img_name'] = a + '/test/' + b
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv', index=False, header=True)

In [61]:
df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv')
df['img_name'] = df['dataset'] + '/images/' + df['img_name']
df = df.drop(columns=['dataset'])
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv', index=False, header=True)

In [7]:
from ultralytics.engine.validator import BaseValidator
from ultralytics.utils.metrics import box_iou, Metric, DetMetrics
import numpy as np
import torch
import pandas as pd

# load txt label


validator = BaseValidator()
validator.iouv = torch.arange(0.5, 1, 0.05)

for result in results:
    metric = DetMetrics()
    label_path = result.path.replace('images', 'labels')[:-4] + '.txt'
    label = torch.from_numpy(np.loadtxt(label_path))
    # turn label into (x1, y1, x2, y2) format
    
    if label.dim() > 1:
        x1 = label[:, 1] - label[:, 3] / 2
        y1 = label[:, 2] - label[:, 4] / 2
        x2 = label[:, 1] + label[:, 3] / 2
        y2 = label[:, 2] + label[:, 4] / 2
        label = torch.stack((label[:, 0], x1, y1, x2, y2), 1)
        iou = box_iou(label[:, 1:].to("cuda:0"), result.boxes.xyxyn.to("cuda:0"))
        tp = validator.match_predictions(result.boxes.data[:,-1], label[:, 0].to("cuda:0"), iou)
        
        # update metric
        tp = tp.detach().cpu().numpy()
        conf = result.boxes.conf.detach().cpu().numpy()
        pre_cls = result.boxes.cls.detach().cpu().numpy()
        target_cls = label[:, 0].detach().cpu().numpy()
    
    else:
        x1 = label[1] - label[3] / 2
        y1 = label[2] - label[4] / 2
        x2 = label[1] + label[3] / 2
        y2 = label[2] + label[4] / 2
        label = torch.tensor([label[0], x1, y1, x2, y2])
        iou = box_iou(label[1:].to("cuda:0").reshape(1, -1), result.boxes.xyxyn.to("cuda:0"))
        tp = validator.match_predictions(result.boxes.data[:,-1], label[0].to("cuda:0").unsqueeze(0), iou)
    
        # update metric
        tp = tp.detach().cpu().numpy()
        conf = result.boxes.conf.detach().cpu().numpy()
        pre_cls = result.boxes.cls.detach().cpu().numpy()
        target_cls = label[0].unsqueeze(0).detach().cpu().numpy()
    
    _, _, p, r, f1, f2, ap50, ap_50_95, unique_classes, p_curve, r_curve, f1_curve, f2_curve, x, prec_values = ap_per_class(tp, conf, pre_cls, target_cls)
    print(f2.mean(), unique_classes)
      

1.0 [1 9]
1.0 [8]
0.0 [2]
1.0 [8]
0.8125 [1 9]
1.0 [6]
0.0 [0]
0.0 [8]
1.0 [7]
0.0 [7]
1.0 [1 9]
0.8333333333333334 [1]
0.0 [2]
0.0 [6]
0.0 [6]
0.0 [5]
0.8861831775579442 [3]
0.0 [8]
0.5 [3]
0.9894106530883233 [1 9]
1.0 [1 9]
0.0 [3]
1.0 [4]
0.0 [9]
0.25 [1 4 8 9]
1.0 [6]
0.955061678002223 [4 9]
1.0 [8]
1.0 [8]
0.0 [2]
1.0 [6 8]
1.0 [8]
0.0 [6]
1.0 [4]
0.9698366958523534 [9]
0.8333333333333334 [9]
1.0 [8]
0.9166666666666667 [4 9]
1.0 [7]
1.0 [6]
0.9784231710625808 [6]
0.0 [6]
1.0 [4]
0.0 [6]
0.0 [6]
0.0 [2]
1.0 [4 9]
0.969202609172539 [4 9]
1.0 [8]
0.0 [2]
0.6682954202760385 [3]
0.0 [6]
0.9784407998647183 [7]
0.8620689655172415 [3]
1.0 [1 9]
1.0 [4 9]
0.0 [2]
1.0 [7]
0.0 [0]
0.5 [1 9]
0.9166666666666667 [6 8]
1.0 [8]
1.0 [6]
0.0 [6]
0.0 [7]
0.9704364934628894 [8]
0.0 [2]
0.0 [5]
0.5 [3]
0.0 [6]
0.0 [2 6]
0.0 [2]
0.0 [6]
1.0 [1]
1.0 [3]
1.0 [6]
0.0 [6 8]
0.0 [6]
0.0 [5]
0.0 [7]
0.8282790301542053 [3]
1.0 [9]
0.0 [2]
0.9166666666666667 [1 9]
1.0 [8]
1.0 [1 9]
1.0 [6]
0.0 [3]
0.0 [2 8]
1.

In [8]:
# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs


In [9]:
import numpy as np
import ast
import pandas as pd

def safe_convert_to_list(s):
    try:
        # Attempt to directly evaluate the string
        return ast.literal_eval(s)
    except SyntaxError:
        # If direct evaluation fails, attempt to manually parse the string
        cleaned_str = s.strip('[]')
        if cleaned_str:  # Check if the string is not empty
            numbers = [float(num) for num in cleaned_str.split() if num not in ['[', ']']]
            return numbers
        else:
            return []


# Load the dataset
data_path = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/merged_new.csv'
data = pd.read_csv(data_path)

# Display the first few rows of the dataframe to understand its structure
data.head()

# Apply the conversion function to the 'f2' column and then compute the mean F2 score
data['f2'] = data['f2'].apply(safe_convert_to_list)
data['mean_f2'] = data['f2'].apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)

# Prepare the dataset for modeling
features = ['visibility', 'exposure', 'objectness_uncertainty', 'weighted_variance_sum', 'weighted_entropy']
# features = ['visibility', 'exposure']
# features = ['objectness_uncertainty', 'weighted_variance_sum', 'weighted_entropy']
X = data[features]
# y = data['mean_f2']
y = data['ap_50_95']

# Display the first few rows of features and target to verify

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=1000), # Increased max_iter for convergence
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    # Predict
    y_pred = model.predict(X_test)
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R2": r2}
    
    if name == 'Decision Tree Regressor':
        # save test resulst to csv
        df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
        df.to_csv(f'/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/test_result.csv', index=False)

results_df = pd.DataFrame(results).T  # Convert results to a DataFrame for better readability
results_df

,MSE,R2
Linear Regression,0.065377,0.131179
Decision Tree Regressor,0.096486,-0.282236
MLP Regressor,0.068393,0.091110
Random Forest Regressor,0.047503,0.368724


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>MSE</th>
      <th>R2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Linear Regression</th>
      <td>0.197254</td>
      <td>0.064327</td>
    </tr>
    <tr>
      <th>Decision Tree Regressor</th>
      <td>0.380059</td>
      <td>-0.802814</td>
    </tr>
    <tr>
      <th>MLP Regressor</th>
      <td>0.193648</td>
      <td>0.081430</td>
    </tr>
    <tr>
      <th>Random Forest Regressor</th>
      <td>0.213226</td>
      <td>-0.011437</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>MSE</th>
      <th>R2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Linear Regression</th>
      <td>0.199501</td>
      <td>0.053668</td>
    </tr>
    <tr>
      <th>Decision Tree Regressor</th>
      <td>0.283149</td>
      <td>-0.343119</td>
    </tr>
    <tr>
      <th>MLP Regressor</th>
      <td>0.200023</td>
      <td>0.051190</td>
    </tr>
    <tr>
      <th>Random Forest Regressor</th>
      <td>0.161334</td>
      <td>0.234711</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>MSE</th>
      <th>R2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Linear Regression</th>
      <td>0.188070</td>
      <td>0.107889</td>
    </tr>
    <tr>
      <th>Decision Tree Regressor</th>
      <td>0.275943</td>
      <td>-0.308935</td>
    </tr>
    <tr>
      <th>MLP Regressor</th>
      <td>0.185389</td>
      <td>0.120607</td>
    </tr>
    <tr>
      <th>Random Forest Regressor</th>
      <td>0.140655</td>
      <td>0.332804</td>
    </tr>
  </tbody>
</table>
</div>

In [35]:
model.feature_importances_

array([    0.16596,     0.16575,     0.17716,     0.26334,     0.22779])

In [112]:
X_train.describe()

,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy
count,4.867000e+03,4867.000000,4.867000e+03,4867.000000,4867.000000
mean,2.796616e-01,0.232434,6.412672e-03,0.144884,0.001363
std,2.144880e-01,0.268423,4.923872e-03,0.072237,0.004153
min,3.222425e-07,0.000000,1.274079e-07,0.050507,0.000000
25%,1.034640e-01,0.016136,2.396813e-03,0.096459,0.000037
50%,2.356180e-01,0.119043,5.491689e-03,0.105285,0.000154
75%,4.143271e-01,0.384620,9.171929e-03,0.203721,0.000790
max,9.695124e-01,0.999207,2.989884e-02,0.479202,0.056565


In [113]:
X_test.describe()

,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy
count,1217.000000,1217.000000,1.217000e+03,1217.000000,1217.000000
mean,0.270493,0.230253,6.240577e-03,0.149462,0.001661
std,0.209810,0.272485,4.841622e-03,0.073080,0.004813
min,0.000420,0.000000,3.902979e-08,0.055160,0.000000
25%,0.093177,0.010945,2.279311e-03,0.097146,0.000037
50%,0.230777,0.109753,5.438593e-03,0.107024,0.000173
75%,0.402377,0.393562,8.849673e-03,0.226471,0.000954
max,0.958951,0.999026,2.272954e-02,0.413786,0.040970


In [15]:
y_train.describe()

count    5651.000000
mean        0.240280
std         0.275259
min         0.000000
25%         0.000000
50%         0.099500
75%         0.466655
max         0.995000
Name: ap_50_95, dtype: float64

In [16]:
y_test.describe()

count    1413.000000
mean        0.236373
std         0.274412
min         0.000000
25%         0.000000
50%         0.099500
75%         0.447750
max         0.995000
Name: ap_50_95, dtype: float64

In [7]:
from pyiqa.archs.niqe_arch import *
from pyiqa.utils import load_file_from_url

/Data4/student_zhihan_data/Anaconda3/envs/yolov8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
default_model_urls = {
    'url': 'https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/niqe_modelparameters.mat',
    'niqe': 'https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/niqe_modelparameters.mat',
    'ilniqe': 'https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/ILNIQE_templateModel.mat',
    'pretrained': '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/model.mat'
}

In [9]:
import cv2
path = '/Data4/student_zhihan_data/data/GC10-DET_brightness_110/test/images/img_01_425000300_00630_jpg.rf.12001adc8b86faf88a47b6aa6f321b91.jpg'
img = cv2.imread(path)
img = torch.from_numpy(img)
img = img.permute(2,0,1).unsqueeze(0)
# calculate_niqe(img, color_space='gray', pretrained_model_path=load_file_from_url(default_model_urls['niqe']))
# calculate_ilniqe(img, color_space='gray', pretrained_model_path=default_model_urls['pretrained'])
calculate_niqe(img, color_space='gray', pretrained_model_path='model.mat')

tensor(1259.6615, dtype=torch.float64)

In [85]:
# load matlab model
from scipy.io import loadmat
covar = loadmat('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/covar.mat')
mean = loadmat('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/mean.mat')
model = loadmat(load_file_from_url(default_model_urls['niqe']))
model['mu_prisparam'] = np.array(mean['mean'])
model['cov_prisparam'] = np.array(covar['covariance'])
# save mat model
import scipy.io
scipy.io.savemat('model.mat', model)

In [81]:
model = loadmat('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/model.mat')
# change model key name 'None' to 'templateModel'
model['templateModel'] = model.pop('None')
scipy.io.savemat('model.mat', model)

In [86]:
loadmat('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/model.mat')

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Feb 14 16:14:01 2024',
 '__version__': '1.0',
 '__globals__': [],
 'mu_prisparam': array([[     2.8918,      1.1237,     0.94071,  -0.0074117,     0.30461,     0.29457,     0.92068,     0.16218,     0.21715,     0.36395,     0.93394,   -0.093931,     0.33724,     0.25074,       0.934,   -0.094519,     0.33749,     0.25046,      2.7161,       1.036,     0.91893,    0.018017,     0.24276,
             0.25803,     0.90293,     0.12705,     0.18806,     0.29925,     0.89492,   -0.053007,     0.26442,     0.21893,     0.89485,   -0.055262,     0.26535,     0.21794]]),
 'cov_prisparam': array([[    0.11128,     0.04245,    0.031441, ...,   -0.003672,    0.022724,    0.018208],
        [    0.04245,    0.022494,    0.013837, ...,  -0.0016413,    0.010588,   0.0086856],
        [   0.031441,    0.013837,   0.0098078, ...,  -0.0011065,   0.0070665,   0.0057277],
        ...,
        [  -0.003672,  -0.0016413,  -0.0011065, ..

In [75]:
loadmat(load_file_from_url(default_model_urls['niqe']))

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Aug 24 17:52:00 2012',
 '__version__': '1.0',
 '__globals__': [],
 'mu_prisparam': array([[     2.6013,      0.9057,     0.81205,    0.090427,     0.13873,     0.20603,     0.81897,    0.062462,     0.15333,     0.19591,     0.82647,   -0.025526,     0.18857,     0.16578,     0.82429,   -0.025361,     0.18724,     0.16505,      2.9695,     0.96123,     0.84935,    0.082383,     0.16132,
             0.22492,     0.85895,    0.055084,     0.17531,     0.21713,     0.87208,   -0.032221,     0.21549,     0.18821,      0.8694,   -0.032326,     0.21474,     0.18678]]),
 'cov_prisparam': array([[    0.45348,    0.096101,    0.082763, ...,  -0.0068539,    0.041395,    0.031916],
        [   0.096101,    0.037112,    0.021553, ...,  -0.0032338,    0.012877,   0.0095948],
        [   0.082763,    0.021553,    0.017707, ...,  -0.0016373,   0.0089932,   0.0069435],
        ...,
        [ -0.0068539,  -0.0032338,  -0.0016373,

In [45]:
# read csv
import pandas as pd
import cv2
df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/merged.csv')
# calculate niqe for each row and add to new column
for idx, row in df.iterrows():
    img = cv2.imread(row['img_name'])
    img = torch.from_numpy(img)
    img = img.permute(2,0,1).unsqueeze(0)
    try:
        niqe = calculate_niqe(img, color_space='gray', pretrained_model_path='/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/model.mat')
        df.loc[idx, 'niqe'] = niqe.item()
    except:
        #set NAN
        niqe = np.nan
        df.loc[idx, 'niqe'] = niqe


/Data4/student_zhihan_data/Anaconda3/envs/yolov8/lib/python3.9/site-packages/pyiqa/matlab_utils/functions.py:167: UserWarning: cov(): degrees of freedom is <= 0 (Triggered internally at ../aten/src/ATen/native/Correlation.cpp:117.)
  return torch.cov(tensor, correction=correction)


In [47]:
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/merged_new.csv', index=False, header=True)

In [74]:
data = df.dropna()

# scale niqe to [0,1]
# data['niqe'] = (data['niqe'] - data['niqe'].min()) / (data['niqe'].max() - data['niqe'].min())
data['niqe'] = data['niqe'] / data['niqe'].max()

/tmp/ipykernel_2718285/2985616201.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['niqe'] = data['niqe'] / data['niqe'].max()


In [75]:
import ast
# Apply the conversion function to the 'f2' column and then compute the mean F2 score
def safe_convert_to_list(s):
    try:
        # Attempt to directly evaluate the string
        return ast.literal_eval(s)
    except SyntaxError:
        # If direct evaluation fails, attempt to manually parse the string
        cleaned_str = s.strip('[]')
        if cleaned_str:  # Check if the string is not empty
            numbers = [float(num) for num in cleaned_str.split() if num not in ['[', ']']]
            return numbers
        else:
            return []
        
data['f2'] = data['f2'].apply(safe_convert_to_list)
data['mean_f2'] = data['f2'].apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)

# Prepare the dataset for modeling
features = ['visibility', 'exposure', 'objectness_uncertainty', 'weighted_variance_sum', 'weighted_entropy', 'niqe']
# features = ['visibility', 'exposure']
# features = ['objectness_uncertainty', 'weighted_variance_sum', 'weighted_entropy']
X = data[features]
y = data['mean_f2']

/tmp/ipykernel_2718285/729834478.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['f2'] = data['f2'].apply(safe_convert_to_list)
/tmp/ipykernel_2718285/729834478.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mean_f2'] = data['f2'].apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)


In [76]:
X.dropna().describe()

,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy,niqe
count,6.083000e+03,6083.000000,6.083000e+03,6083.000000,6083.000000,6083.000000
mean,2.777211e-01,0.231871,6.378834e-03,0.145793,0.001423,0.001273
std,2.134308e-01,0.269062,4.907808e-03,0.072428,0.004295,0.024679
min,3.222425e-07,0.000000,3.902979e-08,0.050507,0.000000,0.000000
25%,1.020507e-01,0.014917,2.376295e-03,0.096630,0.000037,0.000033
50%,2.346791e-01,0.117202,5.489420e-03,0.105499,0.000158,0.000068
75%,4.125931e-01,0.386221,9.104459e-03,0.209953,0.000815,0.000137
max,9.695124e-01,0.999207,2.989884e-02,0.479202,0.056565,1.000000


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=1000), # Increased max_iter for convergence
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    # SVM
    "SVM Regressor": SVR(kernel='linear', C=1.0, epsilon=0.1),
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    # Predict
    y_pred = model.predict(X_test)
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R2": r2}
    
    # if name == 'Decision Tree Regressor':
    #     # save test resulst to csv
    #     temp = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    #     .to_csv(f'/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/test_result.csv', index=False)

results_df = pd.DataFrame(results).T  # Convert results to a DataFrame for better readability
results_df

,MSE,R2
Linear Regression,0.191435,0.100082
Decision Tree Regressor,0.277962,-0.306672
MLP Regressor,0.187570,0.118250
Random Forest Regressor,0.143621,0.324849
SVM Regressor,0.207569,0.024235


In [78]:
model.feature_importances_

array([    0.13651,     0.14236,      0.1451,     0.20377,     0.26094,     0.11132])